<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introcción

#2. Armado del ambiente

In [17]:
url_archivo_1 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2014.txt?raw=true"
url_archivo_2 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2015.txt?raw=true"
url_archivo_3 = "https://github.com/StankoDiego/SOA_EA_2/blob/main/2016.txt?raw=true"

!wget {url_archivo_1} -O "2014.txt"
!wget {url_archivo_2} -O "2015.txt"
!wget {url_archivo_3} -O "2016.txt"

--2020-11-22 19:31:58--  https://github.com/StankoDiego/SOA_EA_2/blob/main/2014.txt?raw=true
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/StankoDiego/SOA_EA_2/raw/main/2014.txt [following]
--2020-11-22 19:31:59--  https://github.com/StankoDiego/SOA_EA_2/raw/main/2014.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/StankoDiego/SOA_EA_2/main/2014.txt [following]
--2020-11-22 19:31:59--  https://raw.githubusercontent.com/StankoDiego/SOA_EA_2/main/2014.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2769731 (2.6M) [t

In [18]:
!ls
!cat 2015.txt

Se han truncado las últimas 5000 líneas del flujo de salida.
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Nafta Grado 2 (Súper),(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Nafta Grado 1 (Común),(m3),0
2015,7,Compañía Mega S.A.,Compras a Empresas del Sector,Aguarras,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Solventes Hexano,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Solventes Aromáticos,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Otros Tipos de Gasoil,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Solventes Alifáticos,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Otros Tipos de Naftas,(m3),0
2015,7,REFINADORA NEUQUINA S.A.,Compras a Empresas del Sector,Propano y Otros C3,(Ton),0
2015,7,Compañía Mega S.A.,Compras a Empresas del Sector,Aeronaftas,(m3),0
2015,7,Compañía Mega S.A.,Compras a Empresas del Sector,Aerokerosene (Jet)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#3. Desarrollo

In [56]:
code = """
// Procesamiento de archivos con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <fstream>
#include <string>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  /*if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }*/

  int cantidad_N = atoi(argv[1]);

   TIEMPO_INI( HTH_AXPY_OMP )

    int number_of_lines = 0;
    std::string line;
    std::ifstream myfile(argv[2]);

    while (std::getline(myfile, line)){
        ++number_of_lines;
    }
    
    std::cout << "Name of file: " << argv[2];
    std::cout << " " << std::endl;
    std::cout << "Number of lines in text file: " << number_of_lines;

  TIEMPO_FIN( HTH_AXPY_OMP )

  //-------------CIERRO_ARCHIVOS------------------
  // --------------------------------------------

}
"""
text_file = open("code_axpy.cpp", "w")
text_file.write(code)
text_file.close()

In [57]:
!g++ -o axpy -fopenmp code_axpy.cpp

In [58]:
%env OMP_NUM_THREADS=3
!./axpy 3 "2015.txt"

env: OMP_NUM_THREADS=3
Name of file: 2015.txt 
Number of lines in text file: 44724

#4. Tabla de pasos

#5. Conclusiones